In [5]:
import os
import pickle
from os.path import join as ojoin

In [3]:
data_dir = os.path.abspath(ojoin('..','..','..','code','fairim','networks'))

In [7]:
test_data = ojoin(data_dir, 'graph_spa_500_0.pickle')
with open(test_data, 'rb') as f:
    data = pickle.load(f)

In [11]:
for node in data.nodes(data=True):
    print(node)

(0, {'region': 'northwest_antelope_valley', 'ethnicity': 'other', 'age': '40-49', 'gender': 'male', 'status': 'obese'})
(1, {'region': 'northwest_antelope_valley', 'ethnicity': 'black', 'age': '30-39', 'gender': 'female', 'status': 'obese'})
(2, {'region': 'northwest_antelope_valley', 'ethnicity': 'white', 'age': '60-64', 'gender': 'male', 'status': 'overweight'})
(3, {'region': 'northwest_antelope_valley', 'ethnicity': 'white', 'age': '30-39', 'gender': 'female', 'status': 'obese'})
(4, {'region': 'northwest_antelope_valley', 'ethnicity': 'white', 'age': '65+', 'gender': 'female', 'status': 'obese'})
(5, {'region': 'northeast_antelope_valley', 'ethnicity': 'latino', 'age': '50-59', 'gender': 'male', 'status': 'overweight'})
(6, {'region': 'northeast_antelope_valley', 'ethnicity': 'latino', 'age': '30-39', 'gender': 'female', 'status': 'overweight'})
(7, {'region': 'northeast_antelope_valley', 'ethnicity': 'black', 'age': '65+', 'gender': 'female', 'status': 'obese'})
(8, {'region': 'n